# Westeros Tutorial Part 1.X - Building a scenario by importing data from Excel

## Scope of this tutorial

This tutorial explains how the Westeros Baseline scenario can be built using data imported from Excel (.xlsx) files. The figure below compares the workflows of the original westeroes tutorial, where data was added using the `add_par()` and `add_set()` functions. The same scenario can also be created by importing data from Excel files, using `ixmp.Scenario.read_excel()` (https://docs.messageix.org/projects/ixmp/en/latest/api.html#ixmp.Scenario.read_excel). The required model data and structure of a MESSAGEix scenario can be imported from either a single or multiple Excel files (as shown below). The advantage of using multiple files, especially for large data sets, is that this provides a more transparent structuring of the underlying workflow, i.e. the scripts related to creating the original xlsx source files and the respective imports will be limited to certain model aspects and therefore will be shorter than if they were to cover all the model data. This means that maintenance and updates can be performed more easily in the future. 

<img src='_static/westeroes_baseline_xlsx_workflow.jpg'>

The approach chosen for the tutorial can be varied and depends very much on user preferences and the specific use-case. We have chosen to split this tutorial into two parts in order to demonstrate how various methods of adding data to the model can be combined.

The first part will create a scenario and basic parameters will be added including the modeling time-horizon and the discount rate. This code is a repetition of that used in the tutorial `westeros_baseline.ipynb`, which will be referred to as the "original" scenario throughout the remainder of this tutorial.

The sceond part will then import various parameters from Excel files, adding technologies, demand and constraints to the model. We have chosen to distribute the import of data across various excel files. This helps to not only outline a possible approach on how to structure the import methods if used for loading large amounts of data into scenarios, but also demonstartes the required data contents of the excel files.

### Online documentation

The documentation of the MESSAGEix framework is available at [https://docs.messageix.org](https://docs.messageix.org)

## Step 1. Setting up the scenario

As in the "original" tutorial, we start by importing all the packages we need.

In [ ]:
import pandas as pd
import ixmp
import message_ix

%matplotlib inline

In [ ]:
# Loading Modeling platform
mp = ixmp.Platform()

In [ ]:
# Creating a new, empty scenario
scenario = message_ix.Scenario(mp, model='Westeros Electrified', 
                               scenario='baseline_xlsx', version='new')

## Model Structure

As in the original tuorial, basic characteristics of the model, including model years, location, and the energy system structure are defined.  We also definie the interest rate.

In [ ]:
# Defining historical and model (optimization) years
history = [690]
model_horizon = [700, 710, 720]
scenario.add_horizon(
    year=history + model_horizon,
    firstmodelyear=model_horizon[0]
)

In [ ]:
# Defining a spatial level and adding a node to it
node = 'Westeros'
scenario.add_spatial_sets({'country': node})

Contrary to the original tutorial for building the Westros model, we only define commodities and levels here. If additional `commodity` or `level` types are required later on, these can also be defined with uploading technologoy data using the excel files. Either method is fine.
Note that we populated the set `technology` and `mode` in the original tutorial. We will address these in the second half of this tutorial.

In [ ]:
# Adding elements to MESSAGEix sets
scenario.add_set("commodity", ["electricity", "light"])
scenario.add_set("level", ["secondary", "final", "useful"])

In [ ]:
# Adding interest rate of 5% per model year
scenario.add_par("interestrate", model_horizon, value=0.05, unit='-')

Note that the data has not yet been commited. This is because at a minimum, the set technology needs to be defined otherwise an error message will be raised when commiting the scenario. Hence, we will continue in this tutorial by adding data, step-by-step, in the process of which commits are automatically made.

## Step 2: Importing data from Excel


### 2.1 Supply and Demand (or Balancing Commodities)

Instead of using the `message_ix.Scenario.add_par()` for adding data to a MESSAGEix parameter, we import data from a xlsx file. The arguments `add_units` has been set to `True`, so that any units which have not yet been specified in the modeling platform will be defined automatically.

In [ ]:
scenario.read_excel("westeroes_baseline_demand.xlsx", add_units=True, commit_steps=False)

Open the above indicated xlsx ("westeroes_baseline_demand.xlsx") file and browse its contents. You will find that it includes two sheets. A sheet `demand` which contains the based on the index structure of the parameters `demand`.

In [ ]:
print(scenario.idx_names('demand'))

In addition, numerical values have been added under the index-name `value` and the units have been specified under the index-name `unit`. These two index names are used for adding numerical values and units across all parameters.

### 2.2 Adding technologies
The following sheets are distinguished in the data input file for the next step ("westeroes_baseline_technology_basic.xlsx"), which adds a basic configuration for technologies.
```
a. mode
b. technology
c. capacity_factor
d. input
e. output
f. technical_lifetime
g. ix_type_mapping
```
The naming convention of the sheets is analogue to the set or parameter to which the data contained within refers to. The exception is the last sheet ("ix_type_mapping"), which is used by the importer to distinguish which sheets are populate sets or parameters. This is important as first sets will need to be populated, to allow the import of parameters. The sets must include all unique entries in the parameters, otherwise if data in the sheet `capacity_factor` is defined for a technology which is not listed in the sheet `technology`, an error will be raised.

In [ ]:
scenario.read_excel("westeroes_baseline_technology_basic.xlsx", add_units=True, commit_steps=False)

In order to view which technologies we have added, we can do the following:

In [ ]:
scenario.set('technology')

We can also look at the specific parameters by:

In [ ]:
scenario.par('capacity_factor')

### 2.3 Technological Diffusion and Contraction
As we have already undertaken an initial definition of technologies, and will now proceed to add additional parameters for already defined technologies, the input data file ("westeroes_baseline_technology_constraint.xlsx") does not contain and sheets relating to sets.

In [ ]:
scenario.read_excel("westeroes_baseline_technology_constraint.xlsx", add_units=True, commit_steps=False)

### 2.4 Defining an Energy Mix (Model Calibration)
In this part, we import data of the historical years.

In [ ]:
scenario.read_excel("westeroes_baseline_technology_historic.xlsx", add_units=True, commit_steps=False)

### 2.5 Investment, Fixed O&M and Varaible O&M Costs
Here, we import the cost data from a separate file.

In [ ]:
scenario.read_excel("westeroes_baseline_technology_economic.xlsx", add_units=True, commit_steps=False)

## Time to Solve the Model

In [ ]:
scenario.set_as_default()

In [ ]:
scenario.solve()

In [ ]:
scenario.var('OBJ')['lvl']

## Plotting Results

Analyzing the results of a scenario after it is solved is called **reporting**.
MESSAGE*ix* includes flexible and customizable message_ix/tutorial for reporting, but here we wish to focus on the results of our example scenario.
We use some custom code to set up some simple plots:

In [ ]:
# Create a Reporter object to describe and carry out reporting
# calculations and operations (like plotting) based on `scenario`
from message_ix.reporting import Reporter

rep = Reporter.from_scenario(scenario)

# Add keys like "plot activity" to describe reporting operations.
# See tutorial/utils/plotting.py
from message_ix.util.tutorial import prepare_plots

prepare_plots(rep)

Now the object `rep` is ready to generate several plots.

(For a detailed introduction to how the `Reporter` works, see `westeros_report.ipynb`.)

### Activity

How much energy is generated in each time period from the different potential sources?

In [ ]:
# Only show a subset of technologies in the follow plots;
# e.g. exclude "bulb" and "grid"
rep.set_filters(t=["coal_ppl", "wind_ppl"])

# Trigger the calculation and plotting
rep.get("plot activity")

### Capacity

How much capacity of each plant is installed in each period?

In [ ]:
# Create a different plot. The same filters are still active.
rep.get("plot capacity")

### Electricity Price

And how much does the electricity cost? These prices are in fact **shadow prices** taken from the **dual variables** of the model solution.
They reflect the marginal cost of electricity generation (i.e., the additional cost of the system for supplying one more unit of
electricity), which is in fact the marginal cost of the most expensive operating generator.  

Note the price drop when the most expensive technology is no longer in the system.

In [ ]:
# Replace the technology filters with a commodity filter;
# show only "light" and not e.g. "electricity".
rep.set_filters(c=["light"])

# Create a price plot
rep.get("plot prices")

## Close the connection to the database

When working with local HSQLDB database instances, you cannot connect to one database from multipe Jupyter notebooks (or processes) at the same time.

If you want to easily switch between notebooks with connections to the same `ixmp` database, you need to close the connection in one notebook before initializing the platform using `ixmp.Platform()` in another notebook.

After having closed the database connection, you can reopen it using
```
mp.open_db()
```

In [ ]:
mp.close_db()

## Congratulations! 

You have built and run your very first *MESSAGEix* model. Welcome to the community!

The next tutorials will introduce you to other features of the framework, including energy system constraints, emissions taxes, and other policy options.

Check us out on Github https://github.com/iiasa/message_ix  
and get in touch with us online https://groups.google.com/forum/message-ix ...